In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.<INSERT_VERSION>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-20 23:23:29--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.75MB/s    in 0.5s    

2021-02-20 23:23:30 (1.75 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Madrid Listings").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/unit-7/listings.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("listings.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+-----+--------------------+-------+---------+-------------------+--------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|   id|                name|host_id|host_name|neighbourhood_group| neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+-----+--------------------+-------+---------+-------------------+--------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
| 6369|Rooftop terrace r...|  13660|    Simon|          Chamartín|Hispanoamérica|40.45628| -3.67763|   Private room|   70|             1|               65| 2019-09-04|             0.56|                             1|              53|
|21853|Bright and airy room|  83531|    Abdel|             Latin

In [6]:
df = df.withColumn("price", df["price"].cast('float'))
df = df.withColumn("calculated_host_listings_count", df["calculated_host_listings_count"].cast('float'))
df = df.withColumn("availability_365", df["availability_365"].cast('float'))

In [7]:
priceGrouped = df.groupBy('neighbourhood_group')
priceGrouped.agg({'price':'Avg'}).show()

+--------------------+------------------+
| neighbourhood_group|        avg(price)|
+--------------------+------------------+
|               Rejas|               1.0|
|            Pacífico|               2.0|
|         Universidad|               1.6|
|Casco Histórico d...|               1.0|
|                Goya|               1.0|
|   Villa de Vallecas|110.62745098039215|
|                José|              null|
|              Tetuán| 160.6714628297362|
|              Amalia|              null|
|           Hortaleza|164.56495468277944|
|            Yolvania|              null|
|           San Diego|               2.0|
|                null|              null|
|            Gabriela|              null|
|  Puente de Vallecas|  85.7677966101695|
|            Chamberí| 124.3288186606471|
|              Latina|126.01628664495114|
|            Simancas|               2.0|
|              Opañel|               2.0|
|       Valdeacederas|               2.0|
+--------------------+------------

In [10]:
roomTypeGrouped = df.groupBy('room_type')
roomTypeAgg = roomTypeGrouped.agg({'availability_365':'Avg'})

In [11]:
roomTypeAgg.orderBy(roomTypeAgg['avg(availability_365)'].desc()).show()

+---------------+---------------------+
|      room_type|avg(availability_365)|
+---------------+---------------------+
|     Hotel room|   220.39622641509433|
|    Shared room|   196.88278388278388|
|Entire home/apt|   161.22735601051542|
|   Private room|   139.05402514041188|
|       -3.71107|                  7.0|
|       -3.67971|                  5.0|
|       40.41064|    1.159999966621399|
|        -3.5852|                  1.0|
|       -3.70819|                  1.0|
|       -3.69901|                  1.0|
|             35|                 null|
|            120|                 null|
|             22|                 null|
|           null|                 null|
|             85|                 null|
|            100|                 null|
|             18|                 null|
|           1200|                 null|
|             43|                 null|
|            300|                 null|
+---------------+---------------------+
only showing top 20 rows



In [12]:
greaterthan_75 = df.filter("price>75")

In [13]:
output = greaterthan_75.orderBy(greaterthan_75['price'].desc())
output.show()

+--------+--------------------+---------+----------------+--------------------+--------------+--------+---------+---------------+------+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|      id|                name|  host_id|       host_name| neighbourhood_group| neighbourhood|latitude|longitude|      room_type| price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+--------+--------------------+---------+----------------+--------------------+--------------+--------+---------+---------------+------+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|27418907|Bastardo Hostel -...|189959417|          Adrian|              Centro|      Justicia|40.42449| -3.69903|   Private room|9999.0|             1|                1| 2019-01-20|             0.12|                          10.0|           351.0|
|3485193

In [ ]:
postgres_url="jdbc:postgresql://<Database URL>:5432/<Database Name>"
config = {"user":"postgres", 
          "password": "<PASSWORD>", 
          "driver":"org.postgresql.Driver"}
output.write.jdbc(postgres_url,table="madridlistings",mode="append", properties=config)